In [ ]:
!pip install selenium

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

print("Libraries imported successfully!")

Libraries imported successfully!


international


In [ ]:
!pip install undetected-chromedriver


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47047 sha256=b95d66d1a64a4d24666f8761da96e05d2f607267f997a94e013f91c63bb76648
  Stored in directory: /root/.cache/pip/wheels/5c/b9/03/4b6e38f019d6170e8c25df2e1e362d7bdf9ff4012df2dc85c0
Successfully built undetected-chromedriver


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_howstat_players(match_type):
    """
    Scrape player data from HowStat.com

    Parameters:
    country (str): Country code (IND, AUS, ENG, etc.)
    match_type (str): Match type (Test, ODI, T20I)

    Returns:
    DataFrame with player data
    """
    url = "https://www.howstat.com/Cricket/Statistics/Players/PlayerCountryList.asp"

    # Headers to mimic a browser
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
        "Referer": "https://www.howstat.com/Cricket/Statistics/Players/PlayerCountryList.asp"
    }

    # Create params for GET request
    form_data = {
        "cboFormat": match_type,     # "T" for Test, "O" for ODI, "W" for T20I
        "cboCountry": "IND",  # Country code (e.g., "IND" for India)
        "chkCurrent": "T"     # Include this for Recent Only!
    }


    # Make the request
    response = requests.post(url, headers=headers, data=form_data)

    if response.status_code != 200:
        print(f"Failed to retrieve data: Status code {response.status_code}")
        return None

    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the main table - based on your screenshot
    tables = soup.find_all('table', {'class': 'TableLined'})
    # print(tables)

    if not tables:
        print("Could not find player table")
        return None

    # Get headers from the first row of the table
    headers = []
    header_row = tables[0].find('tr')
    # print(header_row)
    for th in header_row.find_all('td'):
        headers.append(th.text.strip())
    # print(f"Header: {headers}")

    # Parse player data from table rows
    player_data = []
    for row in tables[0].find_all('tr')[1:]:  # Skip header row
        cells = row.find_all('td')
        if len(cells) > 1:  # Ensure it's a data row
            player = {}
            for i, cell in enumerate(cells):
                if i < len(headers):
                    player[headers[i]] = cell.text

                    # # Extract player link if available
                    # if i == 0 and cell.find('a'):
                    #     player['link'] = cell.find('a').get('href')

            player_data.append(player)

    # Convert to DataFrame
    df = pd.DataFrame(player_data)
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip()

        # 2. Replace all "-" values with 0
        # Note: This step is usually better handled with care for numeric columns
        cols_to_replace_dash = ['Bowl Avg', 'E/R', 'Best']
        for col in cols_to_replace_dash:
            df[col] = df[col].replace('-', '0')
        df['Name'] = df['Name'].str.replace(r'\*$', '', regex=True).str.strip()

        # Remove "*" at the end of "HS" (Highest Score) values
        df['HS'] = df['HS'].str.replace(r'\*$', '', regex=True).str.strip()

        # 3. Convert appropriate columns to numeric
        if match_type=='T':
          numeric_cols = ['Matches', 'Inns', 'Runs', '100s', 'Wkts', '5W', 'Bat Avg', 'Bowl Avg', 'E/R']
        else :
          numeric_cols = ['Matches', 'Inns', 'Runs', '100s', 'Wkts', '4W', 'Bat Avg', 'Bowl Avg', 'E/R']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0)  # Replace NaN with 0

    return df

In [ ]:
test=scrape_howstat_players("T")
t20=scrape_howstat_players("W")
odi=scrape_howstat_players("O")

test.to_csv("test_international.csv",index=False)
t20.to_csv("t20_international.csv",index=False)
odi.to_csv("odi_international.csv",index=False)

Ranji


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/291/ranji-trophy")
    print("Page loaded:", driver.title)

    # Wait much longer for JavaScript to run (crucial!)
    time.sleep(10)  # Give plenty of time for JS to populate the table

    # Custom JavaScript to check if table has data rows
    has_rows = driver.execute_script("""
        var rows = document.querySelectorAll('#stats-body tr');
        return rows.length > 0;
    """)

    if not has_rows:
        print("No data rows found in table. The data might not be loading properly.")

        # Force a click on the Stats tab to ensure it's active
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        time.sleep(5)  # Wait after clicking

    # Use JavaScript to extract the data directly - most reliable method
    data = driver.execute_script("""
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS", "AVG", "BF", "SR", "100", "50", "4S", "6S"];
        var rows = document.querySelectorAll('#stats-body tr');
        var result = [];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            var rowData = {};

            for (var j = 0; j < cells.length; j++) {
                rowData[headers[j]] = cells[j].textContent.trim();
            }

            if (Object.keys(rowData).length > 0) {
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data if data else [])
    print(f"Successfully scraped {len(df)} rows of data")
    # print(df.head())
    # Define the correct column order based on the BCCI website table
    correct_order = [
        "POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS",
        "AVG", "BF", "SR", "100", "50", "4S", "6S"
    ]

    # Reorder the DataFrame columns
    df = df[correct_order]

    # Now display the first few rows
    print(df.head())

    df.to_csv("ranji_runs.csv", index=False)

    if len(df) == 0:
        print("\nDebug info - Document content:")
        print(driver.page_source[:1000] + "...")  # Print first 1000 chars of page source

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
No data rows found in table. The data might not be loading properly.
Successfully scraped 100 rows of data
  POS             PLAYER   TEAM   M   I    R NO    HS     AVG    BF     SR  \
0   1         Y V Rathod    VCA  10  18  960  -   151   53.33  1630  58.89   
1   2  Snehal Kauthankar  GOACA   6   7  949  2  314*  189.80  1323  71.73   
2   3     Shubham Sharma   MPCA   7  12  943  3   240  104.78  1451  64.98   
3   4     Tanmay Agarwal   HYCA   7  12  934  -   177   77.83  1619  57.68   
4   5        Agni Chopra    CAM   5   8  865  1  238*  123.57   987  87.63   

  100 50   4S  6S  
0   5  3   80   4  
1   3  2  100   4  
2   3  4   84  10  
3   4  2   81  11  
4   4  2  123   7  


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/291/ranji-trophy")
    print("Page loaded:", driver.title)

    # Wait for page to load
    time.sleep(10)

    # First check if we need to click on the Stats tab
    try:
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        print("Clicked on Stats tab")
        time.sleep(5)
    except:
        print("Stats tab not found or already active")

    # Then try to click on Most Wickets in the Bowling Records section
    try:

        # Then click on Most Wickets
        most_wickets = driver.find_element(By.XPATH, "//a[contains(text(), 'Most Wickets')]")
        driver.execute_script("arguments[0].click();", most_wickets)
        print("Clicked on Most Wickets")
        time.sleep(10)  # Give it time to load
    except:
        print("Most Wickets link not found or already active")

    # Use document.querySelectorAll directly to get data from the visible table
    data = driver.execute_script("""
        // First, let's check what data we have
        var rows = document.querySelectorAll('table tbody tr');
        console.log("Found " + rows.length + " rows in all tables");

        // Try to get data from any visible table with content
        var result = [];
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            if (cells.length >= 10) {  // Make sure it has enough cells to be our data
                var rowData = {};
                for (var j = 0; j < Math.min(cells.length, headers.length); j++) {
                    rowData[headers[j]] = cells[j].textContent.trim();
                }
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data)
    print(f"Successfully scraped {len(df)-38} rows of bowling data")
    columns = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"]

    # Only use columns that exist in the DataFrame
    available_columns = [col for col in columns if col in df.columns]
    df = df[available_columns]

    # Display the first few rows


    # Save to CSV
    df=df.iloc[38:]
    print(df.head())
    df.to_csv("ranji_wickets.csv", index=False)

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
Clicked on Stats tab
Clicked on Most Wickets
Successfully scraped 138 rows of bowling data
   POS        PLAYER  TEAM   M   I WKTS     R BBI    AVG  ECON     SR 4W 5W
38   1   Harsh Dubey   VCA  10  19   69  1172   6  16.98  2.66  38.17  3  7
39   2    Auqib Nabi  JKCA   8  15   44   613   6  13.93  2.74  30.47  2  6
40   3  Shams Mulani   MCA   9  18   44  1035   6  23.52  2.97  47.40  1  3
41   4     Suchith J   NCA   6  11   43   695   6  16.16  2.39  40.51  3  4
42   5    D A Jadeja   SCA   8  13   40   834   6  20.85  2.98  41.97  2  3


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/289/duleep-trophy")
    print("Page loaded:", driver.title)

    # Wait much longer for JavaScript to run (crucial!)
    time.sleep(10)  # Give plenty of time for JS to populate the table

    # Custom JavaScript to check if table has data rows
    has_rows = driver.execute_script("""
        var rows = document.querySelectorAll('#stats-body tr');
        return rows.length > 0;
    """)

    if not has_rows:
        print("No data rows found in table. The data might not be loading properly.")

        # Force a click on the Stats tab to ensure it's active
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        time.sleep(5)  # Wait after clicking

    # Use JavaScript to extract the data directly - most reliable method
    data = driver.execute_script("""
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS", "AVG", "BF", "SR", "100", "50", "4S", "6S"];
        var rows = document.querySelectorAll('#stats-body tr');
        var result = [];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            var rowData = {};

            for (var j = 0; j < cells.length; j++) {
                rowData[headers[j]] = cells[j].textContent.trim();
            }

            if (Object.keys(rowData).length > 0) {
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data if data else [])
    print(f"Successfully scraped {len(df)} rows of data")
    # print(df.head())
    # Define the correct column order based on the BCCI website table
    correct_order = [
        "POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS",
        "AVG", "BF", "SR", "100", "50", "4S", "6S"
    ]

    # Reorder the DataFrame columns
    df = df[correct_order]

    # Now display the first few rows
    print(df.head())

    df.to_csv("duleep_runs.csv", index=False)

    if len(df) == 0:
        print("\nDebug info - Document content:")
        print(driver.page_source[:1000] + "...")  # Print first 1000 chars of page source

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
No data rows found in table. The data might not be loading properly.
Successfully scraped 67 rows of data
  POS           PLAYER   TEAM  M  I    R NO    HS    AVG   BF     SR 100 50  \
0   1       Ricky Bhui  IND-D  3  6  359  1  119*  71.80  561  63.99   2  1   
1   2     A R Easwaran  IND-B  3  5  309  1  157*  77.25  553  55.87   2  0   
2   3   Shashwat Rawat  IND-A  2  4  256  1   124  85.33  424  60.37   1  2   
3   4      R D Gaikwad  IND-C  3  6  232  -    62  38.67  347  66.85   0  2   
4   5  B Sai Sudharsan  IND-C  3  6  211  -   111  35.17  397  53.14   1  0   

   4S 6S  
0  43  8  
1  31  2  
2  28  1  
3  34  3  
4  29  0  


Duleep trophy

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/289/duleep-trophy")
    print("Page loaded:", driver.title)

    # Wait for page to load
    time.sleep(10)

    # First check if we need to click on the Stats tab
    try:
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        print("Clicked on Stats tab")
        time.sleep(10)
    except:
        print("Stats tab not found or already active")

    # Then try to click on Most Wickets in the Bowling Records section
    try:

        # Then click on Most Wickets
        most_wickets = driver.find_element(By.XPATH, "//a[contains(text(), 'Most Wickets')]")
        driver.execute_script("arguments[0].click();", most_wickets)
        print("Clicked on Most Wickets")
        time.sleep(15)  # Give it time to load
    except:
        print("Most Wickets link not found or already active")

    # Use document.querySelectorAll directly to get data from the visible table
    data = driver.execute_script("""
        // First, let's check what data we have
        var rows = document.querySelectorAll('table tbody tr');
        console.log("Found " + rows.length + " rows in all tables");

        // Try to get data from any visible table with content
        var result = [];
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            if (cells.length >= 10) {  // Make sure it has enough cells to be our data
                var rowData = {};
                for (var j = 0; j < Math.min(cells.length, headers.length); j++) {
                    rowData[headers[j]] = cells[j].textContent.trim();
                }
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data)
    print(f"Successfully scraped {len(df)-38} rows of bowling data")
    columns = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"]

    # Only use columns that exist in the DataFrame
    available_columns = [col for col in columns if col in df.columns]
    df = df[available_columns]

    # Display the first few rows


    # Save to CSV
    df=df.iloc[4:]
    print(df.head())
    df.to_csv("duleep_wickets.csv", index=False)

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
Clicked on Stats tab
Clicked on Most Wickets
Successfully scraped 8 rows of bowling data
  POS          PLAYER   TEAM  M  I WKTS    R BBI    AVG  ECON     SR 4W 5W
4   1   Anshul Kamboj  IND-C  3  5   16  274   8  17.12  3.19  32.18  0  1
5   2    Mukesh Kumar  IND-B  3  6   15  429   4  28.60  4.00  42.80  2  0
6   3   Navdeep Saini  IND-B  3  6   14  356   5  25.42  3.84  39.64  0  1
7   4  Arshdeep Singh  IND-D  3  6   13  273   6  21.00  3.23  38.92  0  1
8   5        V Vyshak  IND-C  3  5   10  249   4  24.90  3.23  46.20  1  0


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/289/duleep-trophy")
    print("Page loaded:", driver.title)

    # Wait much longer for JavaScript to run (crucial!)
    time.sleep(10)  # Give plenty of time for JS to populate the table

    # Custom JavaScript to check if table has data rows
    has_rows = driver.execute_script("""
        var rows = document.querySelectorAll('#stats-body tr');
        return rows.length > 0;
    """)

    if not has_rows:
        print("No data rows found in table. The data might not be loading properly.")

        # Force a click on the Stats tab to ensure it's active
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        time.sleep(5)  # Wait after clicking

    # Use JavaScript to extract the data directly - most reliable method
    data = driver.execute_script("""
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS", "AVG", "BF", "SR", "100", "50", "4S", "6S"];
        var rows = document.querySelectorAll('#stats-body tr');
        var result = [];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            var rowData = {};

            for (var j = 0; j < cells.length; j++) {
                rowData[headers[j]] = cells[j].textContent.trim();
            }

            if (Object.keys(rowData).length > 0) {
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data if data else [])
    print(f"Successfully scraped {len(df)} rows of data")
    # print(df.head())
    # Define the correct column order based on the BCCI website table
    correct_order = [
        "POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS",
        "AVG", "BF", "SR", "100", "50", "4S", "6S"
    ]

    # Reorder the DataFrame columns
    df = df[correct_order]

    # Now display the first few rows
    print(df.head())

    df.to_csv("duleep_runs.csv", index=False)

    if len(df) == 0:
        print("\nDebug info - Document content:")
        print(driver.page_source[:1000] + "...")  # Print first 1000 chars of page source

finally:
    driver.quit()


ZR irani cup

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/290/z-r-irani-cup")
    print("Page loaded:", driver.title)

    # Wait much longer for JavaScript to run (crucial!)
    time.sleep(10)  # Give plenty of time for JS to populate the table

    # Custom JavaScript to check if table has data rows
    has_rows = driver.execute_script("""
        var rows = document.querySelectorAll('#stats-body tr');
        return rows.length > 0;
    """)

    if not has_rows:
        print("No data rows found in table. The data might not be loading properly.")

        # Force a click on the Stats tab to ensure it's active
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        time.sleep(5)  # Wait after clicking

    # Use JavaScript to extract the data directly - most reliable method
    data = driver.execute_script("""
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS", "AVG", "BF", "SR", "100", "50", "4S", "6S"];
        var rows = document.querySelectorAll('#stats-body tr');
        var result = [];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            var rowData = {};

            for (var j = 0; j < cells.length; j++) {
                rowData[headers[j]] = cells[j].textContent.trim();
            }

            if (Object.keys(rowData).length > 0) {
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data if data else [])
    print(f"Successfully scraped {len(df)} rows of data")
    # print(df.head())
    # Define the correct column order based on the BCCI website table
    correct_order = [
        "POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS",
        "AVG", "BF", "SR", "100", "50", "4S", "6S"
    ]

    # Reorder the DataFrame columns
    df = df[correct_order]

    # Now display the first few rows
    print(df.head())

    df.to_csv("zrirani_runs.csv", index=False)

    if len(df) == 0:
        print("\nDebug info - Document content:")
        print(driver.page_source[:1000] + "...")  # Print first 1000 chars of page source

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
No data rows found in table. The data might not be loading properly.
Successfully scraped 22 rows of data
  POS             PLAYER TEAM  M  I    R NO    HS     AVG   BF     SR 100 50  \
0   1           S N Khan  MCA  1  2  239  1  222*  239.00  322  74.22   1  0   
1   2       A R Easwaran  ROI  1  1  191  -   191  191.00  292  65.41   1  0   
2   3      Tanush Kotian  MCA  1  2  178  1  114*  178.00  274  64.96   1  1   
3   4     Ajinkya Rahane  MCA  1  2  106  -    97   53.00  261  40.61   0  1   
4   5  Dhruv Chand Jurel  ROI  1  1   93  -    93   93.00  121  76.85   0  1   

   4S 6S  
0  25  4  
1  16  1  
2  16  1  
3   8  1  
4  13  1  


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/290/z-r-irani-cup")
    print("Page loaded:", driver.title)

    # Wait for page to load
    time.sleep(10)

    # First check if we need to click on the Stats tab
    try:
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        print("Clicked on Stats tab")
        time.sleep(10)
    except:
        print("Stats tab not found or already active")

    # Then try to click on Most Wickets in the Bowling Records section
    try:

        # Then click on Most Wickets
        most_wickets = driver.find_element(By.XPATH, "//a[contains(text(), 'Most Wickets')]")
        driver.execute_script("arguments[0].click();", most_wickets)
        print("Clicked on Most Wickets")
        time.sleep(15)  # Give it time to load
    except:
        print("Most Wickets link not found or already active")

    # Use document.querySelectorAll directly to get data from the visible table
    data = driver.execute_script("""
        // First, let's check what data we have
        var rows = document.querySelectorAll('table tbody tr');
        console.log("Found " + rows.length + " rows in all tables");

        // Try to get data from any visible table with content
        var result = [];
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            if (cells.length >= 10) {  // Make sure it has enough cells to be our data
                var rowData = {};
                for (var j = 0; j < Math.min(cells.length, headers.length); j++) {
                    rowData[headers[j]] = cells[j].textContent.trim();
                }
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data)
    print(f"Successfully scraped {len(df)} rows of bowling data")
    columns = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"]

    # Only use columns that exist in the DataFrame
    available_columns = [col for col in columns if col in df.columns]
    df = df[available_columns]

    # Display the first few rows


    # Save to CSV
    # df=df.iloc[4:]
    print(df.head())
    df.to_csv("zrirani_wickets.csv", index=False)

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
Clicked on Stats tab
Clicked on Most Wickets
Successfully scraped -25 rows of bowling data
  POS         PLAYER TEAM  M  I WKTS    R BBI    AVG  ECON     SR 4W 5W
0   1   Saransh Jain  ROI  1  2    7  203   6  29.00  4.14  42.00  0  1
1   2   Mukesh Kumar  ROI  1  2    5  135   5  27.00  4.09  39.60  0  1
2   3  Tanush Kotian  MCA  1  1    3  101   3  33.66  3.74  54.00  0  0
3   4   Shams Mulani  MCA  1  1    3  122   3  40.66  3.05  80.00  0  0
4   5  Mohit Avasthi  MCA  1  1    2   97   2  48.50  4.40  66.00  0  0


https://www.bcci.tv/domestic/292/syed-mushtaq-ali-trophy

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/292/syed-mushtaq-ali-trophy")
    print("Page loaded:", driver.title)

    # Wait much longer for JavaScript to run (crucial!)
    time.sleep(10)  # Give plenty of time for JS to populate the table

    # Custom JavaScript to check if table has data rows
    has_rows = driver.execute_script("""
        var rows = document.querySelectorAll('#stats-body tr');
        return rows.length > 0;
    """)

    if not has_rows:
        print("No data rows found in table. The data might not be loading properly.")

        # Force a click on the Stats tab to ensure it's active
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        time.sleep(5)  # Wait after clicking

    # Use JavaScript to extract the data directly - most reliable method
    data = driver.execute_script("""
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS", "AVG", "BF", "SR", "100", "50", "4S", "6S"];
        var rows = document.querySelectorAll('#stats-body tr');
        var result = [];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            var rowData = {};

            for (var j = 0; j < cells.length; j++) {
                rowData[headers[j]] = cells[j].textContent.trim();
            }

            if (Object.keys(rowData).length > 0) {
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data if data else [])
    print(f"Successfully scraped {len(df)} rows of data")
    # print(df.head())
    # Define the correct column order based on the BCCI website table
    correct_order = [
        "POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS",
        "AVG", "BF", "SR", "100", "50", "4S", "6S"
    ]

    # Reorder the DataFrame columns
    df = df[correct_order]

    # Now display the first few rows
    print(df.head())

    df.to_csv("mushtaq_runs_t20.csv", index=False)

    if len(df) == 0:
        print("\nDebug info - Document content:")
        print(driver.page_source[:1000] + "...")  # Print first 1000 chars of page source

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
No data rows found in table. The data might not be loading properly.
Successfully scraped 100 rows of data
  POS          PLAYER  TEAM   M  I    R NO    HS    AVG   BF      SR 100 50  \
0   1  Ajinkya Rahane   MCA   9  8  469  -    98  58.63  285  164.56   0  5   
1   2   Rajat Patidar  MPCA  10  9  428  2   81*  61.14  230  186.08   0  5   
2   3          S Gani  BICA   7  7  353  1  120*  58.83  248  142.33   1  2   
3   4    Shreyas Iyer   MCA   9  8  345  1  130*  49.29  183  188.52   1  1   
4   5       Karan Lal   CAB   9  9  338  1   94*  42.25  209  161.72   0  2   

   4S  6S  
0  46  19  
1  32  31  
2  41  10  
3  31  22  
4  35  18  


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/292/syed-mushtaq-ali-trophy")
    print("Page loaded:", driver.title)

    # Wait for page to load
    time.sleep(10)

    # First check if we need to click on the Stats tab
    try:
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        print("Clicked on Stats tab")
        time.sleep(10)
    except:
        print("Stats tab not found or already active")

    # Then try to click on Most Wickets in the Bowling Records section
    try:

        # Then click on Most Wickets
        most_wickets = driver.find_element(By.XPATH, "//a[contains(text(), 'Most Wickets')]")
        driver.execute_script("arguments[0].click();", most_wickets)
        print("Clicked on Most Wickets")
        time.sleep(15)  # Give it time to load
    except:
        print("Most Wickets link not found or already active")

    # Use document.querySelectorAll directly to get data from the visible table
    data = driver.execute_script("""
        // First, let's check what data we have
        var rows = document.querySelectorAll('table tbody tr');
        console.log("Found " + rows.length + " rows in all tables");

        // Try to get data from any visible table with content
        var result = [];
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            if (cells.length >= 10) {  // Make sure it has enough cells to be our data
                var rowData = {};
                for (var j = 0; j < Math.min(cells.length, headers.length); j++) {
                    rowData[headers[j]] = cells[j].textContent.trim();
                }
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data)
    print(f"Successfully scraped {len(df)} rows of bowling data")
    columns = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"]

    # Only use columns that exist in the DataFrame
    available_columns = [col for col in columns if col in df.columns]
    df = df[available_columns]

    # Display the first few rows


    # Save to CSV
    df=df.iloc[38:]
    print(df.head())
    df.to_csv("mushtaq_wickets_t20.csv", index=False)

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
Clicked on Stats tab
Clicked on Most Wickets
Successfully scraped 138 rows of bowling data
   POS                 PLAYER  TEAM   M   I WKTS    R BBI    AVG   ECON  \
38   1    Jagjit Singh Sandhu  UTCA   7   7   18  229   5  12.72   8.48   
39   2  Kumar Kartikeya Singh  MPCA  10  10   17  290   3  17.05   7.63   
40   3          M G Choudhary  MHCA   6   6   15  194   4  12.93   8.81   
41   4         Shardul Thakur   MCA   9   9   15  368   4  24.53  10.51   
42   5          Shreyas Gopal  KSCA   7   7   14  166   5  11.85   6.14   

       SR 4W 5W  
38   9.00  1  1  
39  13.41  0  0  
40   8.80  1  0  
41  14.00  1  0  
42  11.57  1  1  


https://www.bcci.tv/domestic/293/vijay-hazare-trophy

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/293/vijay-hazare-trophy")
    print("Page loaded:", driver.title)

    # Wait much longer for JavaScript to run (crucial!)
    time.sleep(10)  # Give plenty of time for JS to populate the table

    # Custom JavaScript to check if table has data rows
    has_rows = driver.execute_script("""
        var rows = document.querySelectorAll('#stats-body tr');
        return rows.length > 0;
    """)

    if not has_rows:
        print("No data rows found in table. The data might not be loading properly.")

        # Force a click on the Stats tab to ensure it's active
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        time.sleep(5)  # Wait after clicking

    # Use JavaScript to extract the data directly - most reliable method
    data = driver.execute_script("""
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS", "AVG", "BF", "SR", "100", "50", "4S", "6S"];
        var rows = document.querySelectorAll('#stats-body tr');
        var result = [];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            var rowData = {};

            for (var j = 0; j < cells.length; j++) {
                rowData[headers[j]] = cells[j].textContent.trim();
            }

            if (Object.keys(rowData).length > 0) {
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data if data else [])
    print(f"Successfully scraped {len(df)} rows of data")
    # print(df.head())
    # Define the correct column order based on the BCCI website table
    correct_order = [
        "POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS",
        "AVG", "BF", "SR", "100", "50", "4S", "6S"
    ]

    # Reorder the DataFrame columns
    df = df[correct_order]

    # Now display the first few rows
    print(df.head())

    df.to_csv("vijay_runs.csv", index=False)

    if len(df) == 0:
        print("\nDebug info - Document content:")
        print(driver.page_source[:1000] + "...")  # Print first 1000 chars of page source

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
No data rows found in table. The data might not be loading properly.
Successfully scraped 100 rows of data
  POS             PLAYER  TEAM   M   I    R NO    HS     AVG   BF      SR 100  \
0   1         Karun Nair   VCA   9   8  779  6  163*  389.50  628  124.04   5   
1   2    Mayank Agarawal  KSCA  10  10  651  3  139*   93.00  605  107.60   4   
2   3           S A Veer  MHCA   9   9  520  3   155   86.67  523   99.42   2   
3   4  Prabhsimran Singh   PCA   8   8  498  2  150*   83.00  387  128.68   3   
4   5       Dhruv Shorey   VCA   8   8  494  1  118*   70.57  533   92.68   3   

  50  4S  6S  
0  1  93  16  
1  1  72  18  
2  2  58  12  
3  0  53  24  
4  1  56   8  


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/293/vijay-hazare-trophy")
    print("Page loaded:", driver.title)

    # Wait for page to load
    time.sleep(10)

    # First check if we need to click on the Stats tab
    try:
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        print("Clicked on Stats tab")
        time.sleep(10)
    except:
        print("Stats tab not found or already active")

    # Then try to click on Most Wickets in the Bowling Records section
    try:

        # Then click on Most Wickets
        most_wickets = driver.find_element(By.XPATH, "//a[contains(text(), 'Most Wickets')]")
        driver.execute_script("arguments[0].click();", most_wickets)
        print("Clicked on Most Wickets")
        time.sleep(15)  # Give it time to load
    except:
        print("Most Wickets link not found or already active")

    # Use document.querySelectorAll directly to get data from the visible table
    data = driver.execute_script("""
        // First, let's check what data we have
        var rows = document.querySelectorAll('table tbody tr');
        console.log("Found " + rows.length + " rows in all tables");

        // Try to get data from any visible table with content
        var result = [];
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            if (cells.length >= 10) {  // Make sure it has enough cells to be our data
                var rowData = {};
                for (var j = 0; j < Math.min(cells.length, headers.length); j++) {
                    rowData[headers[j]] = cells[j].textContent.trim();
                }
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data)
    print(f"Successfully scraped {len(df)} rows of bowling data")
    columns = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"]

    # Only use columns that exist in the DataFrame
    available_columns = [col for col in columns if col in df.columns]
    df = df[available_columns]

    # Display the first few rows


    # Save to CSV
    df=df.iloc[38:]
    print(df.head())
    df.to_csv("vijay_wickets.csv", index=False)

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
Clicked on Stats tab
Clicked on Most Wickets
Successfully scraped 138 rows of bowling data
   POS           PLAYER  TEAM   M   I WKTS    R BBI    AVG  ECON     SR 4W 5W
38   1   Arshdeep Singh   PCA   7   7   20  365   5  18.25  5.62  19.45  2  1
39   2            Varun  TNCA   7   6   18  219   5  12.16  4.36  16.72  0  2
40   3        Koushik V  KSCA  10  10   18  440   5  24.44  4.52  32.44  1  1
41   4    Shreyas Gopal  KSCA  10  10   18  491   4  27.27  4.98  32.83  2  0
42   5  Abhilash Shetty  KSCA   7   7   17  307   5  18.05  5.18  20.88  1  1


https://www.bcci.tv/domestic/295/mens-u23-state-a-trophy

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/295/mens-u23-state-a-trophy")
    print("Page loaded:", driver.title)

    # Wait much longer for JavaScript to run (crucial!)
    time.sleep(10)  # Give plenty of time for JS to populate the table

    # Custom JavaScript to check if table has data rows
    has_rows = driver.execute_script("""
        var rows = document.querySelectorAll('#stats-body tr');
        return rows.length > 0;
    """)

    if not has_rows:
        print("No data rows found in table. The data might not be loading properly.")

        # Force a click on the Stats tab to ensure it's active
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        time.sleep(5)  # Wait after clicking

    # Use JavaScript to extract the data directly - most reliable method
    data = driver.execute_script("""
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS", "AVG", "BF", "SR", "100", "50", "4S", "6S"];
        var rows = document.querySelectorAll('#stats-body tr');
        var result = [];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            var rowData = {};

            for (var j = 0; j < cells.length; j++) {
                rowData[headers[j]] = cells[j].textContent.trim();
            }

            if (Object.keys(rowData).length > 0) {
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data if data else [])
    print(f"Successfully scraped {len(df)} rows of data")
    # print(df.head())
    # Define the correct column order based on the BCCI website table
    correct_order = [
        "POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS",
        "AVG", "BF", "SR", "100", "50", "4S", "6S"
    ]

    # Reorder the DataFrame columns
    df = df[correct_order]

    # Now display the first few rows
    print(df.head())

    df.to_csv("u23_runs_ODI.csv", index=False)

    if len(df) == 0:
        print("\nDebug info - Document content:")
        print(driver.page_source[:1000] + "...")  # Print first 1000 chars of page source

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
No data rows found in table. The data might not be loading properly.
Successfully scraped 100 rows of data
  POS             PLAYER  TEAM  M  I    R NO    HS     AVG   BF      SR 100  \
0   1         Arsh Ranga   HCA  9  9  840  2   230  120.00  588  142.85   4   
1   2       Sameer Rizvi  UPCA  6  6  728  3  202*  242.67  422  172.51   4   
2   3         Himateja K  HYCA  6  6  625  2  164*  156.25  499  125.25   3   
3   4  Yashvardhan Dalal   HCA  9  9  582  2   165   83.14  479  121.50   3   
4   5           Gsp Teja   ACA  9  9  576  1   152   72.00  615   93.65   2   

  50  4S  6S  
0  3  69  51  
1  0  52  62  
2  3  67  18  
3  2  59  26  
4  4  57   6  


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/295/mens-u23-state-a-trophy")
    print("Page loaded:", driver.title)

    # Wait for page to load
    time.sleep(10)

    # First check if we need to click on the Stats tab
    try:
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        print("Clicked on Stats tab")
        time.sleep(10)
    except:
        print("Stats tab not found or already active")

    # Then try to click on Most Wickets in the Bowling Records section
    try:

        # Then click on Most Wickets
        most_wickets = driver.find_element(By.XPATH, "//a[contains(text(), 'Most Wickets')]")
        driver.execute_script("arguments[0].click();", most_wickets)
        print("Clicked on Most Wickets")
        time.sleep(15)  # Give it time to load
    except:
        print("Most Wickets link not found or already active")

    # Use document.querySelectorAll directly to get data from the visible table
    data = driver.execute_script("""
        // First, let's check what data we have
        var rows = document.querySelectorAll('table tbody tr');
        console.log("Found " + rows.length + " rows in all tables");

        // Try to get data from any visible table with content
        var result = [];
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            if (cells.length >= 10) {  // Make sure it has enough cells to be our data
                var rowData = {};
                for (var j = 0; j < Math.min(cells.length, headers.length); j++) {
                    rowData[headers[j]] = cells[j].textContent.trim();
                }
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data)
    print(f"Successfully scraped {len(df)} rows of bowling data")
    columns = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"]

    # Only use columns that exist in the DataFrame
    available_columns = [col for col in columns if col in df.columns]
    df = df[available_columns]

    # Display the first few rows


    # Save to CSV
    df=df.iloc[37:]
    print(df.head())
    df.to_csv("u23_wickets_ODI.csv", index=False)

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
Clicked on Stats tab
Clicked on Most Wickets
Successfully scraped 137 rows of bowling data
   POS          PLAYER  TEAM  M  I WKTS    R BBI    AVG  ECON     SR 4W 5W
37   1          Aniket  HPCA  9  9   25  410   4  16.40  5.56  17.68  3  0
38   2   Piyush Dahiya   HCA  9  9   23  411   4  17.86  5.23  20.47  3  0
39   3  Rounak Waghela  DDCA  7  7   22  225   8  10.22  3.58  17.13  0  2
40   4    Anuj Thakral   HCA  9  9   20  415   4  20.75  5.46  22.80  2  0
41   5    Jay Malusare  GUCA  8  8   19  386   4  20.31  5.58  21.84  1  0


https://www.bcci.tv/domestic/297/cooch-behar-trophy

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/297/cooch-behar-trophy")
    print("Page loaded:", driver.title)

    # Wait much longer for JavaScript to run (crucial!)
    time.sleep(10)  # Give plenty of time for JS to populate the table

    # Custom JavaScript to check if table has data rows
    has_rows = driver.execute_script("""
        var rows = document.querySelectorAll('#stats-body tr');
        return rows.length > 0;
    """)

    if not has_rows:
        print("No data rows found in table. The data might not be loading properly.")

        # Force a click on the Stats tab to ensure it's active
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        time.sleep(5)  # Wait after clicking

    # Use JavaScript to extract the data directly - most reliable method
    data = driver.execute_script("""
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS", "AVG", "BF", "SR", "100", "50", "4S", "6S"];
        var rows = document.querySelectorAll('#stats-body tr');
        var result = [];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            var rowData = {};

            for (var j = 0; j < cells.length; j++) {
                rowData[headers[j]] = cells[j].textContent.trim();
            }

            if (Object.keys(rowData).length > 0) {
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data if data else [])
    print(f"Successfully scraped {len(df)} rows of data")
    # print(df.head())
    # Define the correct column order based on the BCCI website table
    correct_order = [
        "POS", "PLAYER", "TEAM", "M", "I", "R", "NO", "HS",
        "AVG", "BF", "SR", "100", "50", "4S", "6S"
    ]

    # Reorder the DataFrame columns
    df = df[correct_order]

    # Now display the first few rows
    print(df.head())

    df.to_csv("cooch_runs_ODI.csv", index=False)

    if len(df) == 0:
        print("\nDebug info - Document content:")
        print(driver.page_source[:1000] + "...")  # Print first 1000 chars of page source

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
No data rows found in table. The data might not be loading properly.
Successfully scraped 100 rows of data
  POS               PLAYER  TEAM  M   I    R NO   HS     AVG    BF     SR 100  \
0   1       Nitya J Pandya   BCA  6   8  881  -  265  110.13  1182  74.53   4   
1   2              Ragavan   CAP  6   9  782  -  232   86.89  1221  64.04   4   
2   3  Rudra Pritesh Patel  GUCA  8  13  757  2  208   68.82  1091  69.38   3   
3   4          Prithvi Raj  BICA  5   9  734  1  163   91.75   904  81.19   3   
4   5          Smit Rathva   BCA  6   8  733  -  215   91.63  1104  66.39   3   

  50   4S  6S  
0  0  124   8  
1  0   80  23  
2  2   88   7  
3  3  104  11  
4  2  100   6  


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.add_argument("--headless")  # Remove if you want to see the browser

driver = webdriver.Firefox(options=options)

try:
    # Load the page
    driver.get("https://www.bcci.tv/domestic/297/cooch-behar-trophy")
    print("Page loaded:", driver.title)

    # Wait for page to load
    time.sleep(10)

    # First check if we need to click on the Stats tab
    try:
        stats_tab = driver.find_element(By.CSS_SELECTOR, 'button[data-bs-target="#pills-stats"]')
        driver.execute_script("arguments[0].click();", stats_tab)
        print("Clicked on Stats tab")
        time.sleep(10)
    except:
        print("Stats tab not found or already active")

    # Then try to click on Most Wickets in the Bowling Records section
    try:

        # Then click on Most Wickets
        most_wickets = driver.find_element(By.XPATH, "//a[contains(text(), 'Most Wickets')]")
        driver.execute_script("arguments[0].click();", most_wickets)
        print("Clicked on Most Wickets")
        time.sleep(15)  # Give it time to load
    except:
        print("Most Wickets link not found or already active")

    # Use document.querySelectorAll directly to get data from the visible table
    data = driver.execute_script("""
        // First, let's check what data we have
        var rows = document.querySelectorAll('table tbody tr');
        console.log("Found " + rows.length + " rows in all tables");

        // Try to get data from any visible table with content
        var result = [];
        var headers = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"];

        for (var i = 0; i < rows.length; i++) {
            var row = rows[i];
            var cells = row.querySelectorAll('td');
            if (cells.length >= 10) {  // Make sure it has enough cells to be our data
                var rowData = {};
                for (var j = 0; j < Math.min(cells.length, headers.length); j++) {
                    rowData[headers[j]] = cells[j].textContent.trim();
                }
                result.push(rowData);
            }
        }

        return result;
    """)

    # Create DataFrame
    df = pd.DataFrame(data)
    print(f"Successfully scraped {len(df)} rows of bowling data")
    columns = ["POS", "PLAYER", "TEAM", "M", "I", "WKTS", "R", "BBI", "AVG", "ECON", "SR", "4W", "5W"]

    # Only use columns that exist in the DataFrame
    available_columns = [col for col in columns if col in df.columns]
    df = df[available_columns]

    # Display the first few rows


    # Save to CSV
    df=df.iloc[36:]
    print(df.head())
    df.to_csv("cooch_wickets_ODI.csv", index=False)

finally:
    driver.quit()


Page loaded: The Board of Control for Cricket in India
Clicked on Stats tab
Clicked on Most Wickets
Successfully scraped 136 rows of bowling data
   POS          PLAYER  TEAM  M   I WKTS    R BBI    AVG  ECON     SR 4W 5W
36   1  Hemchudeshan J  TNCA  8  15   47  716   5  15.23  2.61  34.89  4  3
37   2  Khilan A Patel  GUCA  8  15   45  984   6  21.86  2.88  45.53  1  5
38   3     Arkajit Roy   TCA  6  11   38  379   6   9.97  2.39  25.02  2  4
39   4     Akshu Bajwa  UPCA  8  15   38  759   8  19.97  2.80  42.71  1  3
40   5      Bk Kishore  TNCA  9  17   38  784   5  20.63  2.85  43.36  3  2
